# DLC/ANIPOSE PIPELINE

Automatically converts fly videos to estimated 3D coordinates using DeepLabCut and Anipose

In [8]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from dlc import analyze_new
from preprocess import gen_anipose_files, clean_dfs 
import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### STEP 0: Configuration

- Add filepath to DLC network config files (e.g `dlc_networks.yml`)
- Add filepath to folder with experiment videos

In [9]:
# Filepath to the geontype folder containing videos
videos = r"C:\Users\ryabinkyj\Documents\testanalyze\RawData\BIN-1"
videos = Path(videos)

p_networks = Path(r'./common_files/dlc_networks.yml')

### STEP 1: Running DeepLabCut

This will run on a directory with fly video files and generate DLC pose estimations, outputting to the same directory.

##### Processing done:
- DeepLabCut `analyze_videos`
- DeepLabCut `filterpredictions`

In [ ]:
analyze_new(videos, p_networks)

### STEP 2: Preprocessing for anipose

##### Processing done:
- Fix points
- Remove columns
- Rename (GenotypeFly#-CamName)
- Convert to .h5
- Generate anipose file structure

In [11]:
# TODO: will probably handle all 3 below from a single directory
error = False

# find all the CSVs that DLC generated
processed_csvs = []
parent_dir = ""
for p_csv in utils.get_csvs(videos):
    parent_dir = p_csv.parent.parent.parent

    # Pick current generated, filtered CSV 
    
    # TODO: also check for cam name and model name
    csv_name = p_csv.stem
    if "filtered" not in csv_name:
        continue

    # Fix points, remove columns 
    csv_df = clean_dfs(p_csv)
    
    processed_csvs.append((csv_df, p_csv))

# Generate anipose file structure
p_calibration_target = Path(r"./common_files/calibration_target.yml") # calibration target config file 
p_calibration_timeline = Path(r"./common_files/calib_timeline.yml") # calibration timeline config file
p_gcam_dummy = Path(r"./GenotypeFly-G.h5") # Dummy file for G camera (top-down view)
calibration_type = utils.get_calibration_type(p_calibration_target, parent_dir)
if calibration_type == 'fly':
    p_anipose_config = Path(r"./common_files/config_fly.toml") # anipose config file
elif calibration_type == 'board':
    p_anipose_config = Path(r"./common_files/config_board.toml") # anipose config file
else:
    print(f"[ERROR] Invalid calibration type or calibration type not specified in {p_calibration_target}")
    error = True

# check that all the files exist
if not p_calibration_target.exists():
    print("[ERROR] `calibration_target.yml` does not exist.")
    error = True
if not p_calibration_timeline.exists():
    print("[ERROR] `calib_timeline.yml` does not exist.")
    error = True
if not p_gcam_dummy.exists():
    print("[ERROR] `GenotypeFly-G.h5` (G camera dummy file) does not exist.")
    error = True

if not error:
    print("[INFO] Generating anipose files...")
    gen_anipose_files(parent_dir, p_networks, p_anipose_config, p_calibration_target, p_calibration_timeline, processed_csvs, p_gcam_dummy)

    print('Finished preprocessing...')
else:
    print("Terminated due to error")

AttributeError: 'str' object has no attribute 'parents'